[![Jupyter Notebook](https://img.shields.io/badge/Jupyter%20Notebook-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/scrna.ipynb)

# Validate & register scRNA-seq datasets

This illustrates how to manage scRNA-seq datasets in absence of a [custom schema](https://lamin.ai/docs/schemas).

```{toctree}
:maxdepth: 1
:hidden:

scrna2
```

In [ ]:
!lamin init --storage ./test-scrna --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

## Human immune cells: Conde22

In [ ]:
lb.settings.species = "human"

### Transform ![](https://img.shields.io/badge/Transform-10b981)

(Here we skip steps of data transformations, which often includes filtering, normalizing, or formatting data.)

Let’s look at a scRNA-seq count matrix in form of an AnnData object:

In [ ]:
conde22 = ln.dev.datasets.anndata_human_immune_cells(
    populate_registries=True  # pre-populate registries to simulate an used instance
)

In [ ]:
conde22

### Validate ![](https://img.shields.io/badge/Validate-10b981)

#### Validate genes in `.var`

In [ ]:
lb.Gene.validate(conde22.var.index, lb.Gene.ensembl_gene_id);

#### Validate metadata in `.obs`

In [ ]:
validated = ln.Feature.validate(conde22.obs.columns)

1 feature is not validated: donor

In [ ]:
conde22.obs.loc[:, ~validated].head()

Let's register it:

In [ ]:
features = ln.Feature.from_df(conde22.obs)

In [ ]:
ln.save(features)

All metadata columns are now validated:

In [ ]:
ln.Feature.validate(conde22.obs.columns);

In [ ]:
lb.CellType.validate(conde22.obs.cell_type)
lb.ExperimentalFactor.validate(conde22.obs.assay)
lb.Tissue.validate(conde22.obs.tissue);

As neither the core schema nor `lnschema_bionty` have a `Donor` table, we're using `Label` to track donor ids.

Donor labels are not validated, so we register them:

In [ ]:
ln.Label.validate(conde22.obs["donor"]);

In [ ]:
donors = [ln.Label(name=name) for name in conde22.obs["donor"].unique()]
ln.save(donors)

In [ ]:
ln.Label.validate(conde22.obs["donor"]);

#### Validate external metadata

In addition to what’s already in the file, we’d like to link this file with external features:

In [ ]:
ln.Feature.validate("species")
lb.Species.validate("human");

In [ ]:
modalities = ln.Modality.lookup()
modalities.rna

Let’s try standardizing synonyms that can’t be validated:

In [ ]:
ln.Feature.validate("assay")
lb.ExperimentalFactor.validate("scRNA-seq");

In [ ]:
scrna_name = lb.ExperimentalFactor.standardize("scRNA-seq")
scrna_name

In [ ]:
lb.ExperimentalFactor.validate(scrna_name);

### Register ![](https://img.shields.io/badge/Register-10b981) 

#### Register data

When we create a File object from an AnnData, we’ll automatically link its feature sets and get information about unmapped categories:

In [ ]:
file = ln.File.from_anndata(
    conde22, description="Conde22", var_ref=lb.Gene.ensembl_gene_id
)

In [ ]:
file.save()

The file has the following 3 linked feature sets:

file.features

You can further annotate your feature set with modality:

In [ ]:
var_feature_set = file.features.get_feature_set("var")
var_feature_set.modality = modalities.rna
var_feature_set.save()

#### Link metadata

Let's now link observational metadata by adding labels to corresponding features.

In [ ]:
file.add_labels(lb.settings.species, feature="species")
file.add_labels(lb.ExperimentalFactor.filter(name=scrna_name).one(), feature="assay")

In [ ]:
cell_types = lb.CellType.from_values(conde22.obs.cell_type, field="name")
efs = lb.ExperimentalFactor.from_values(conde22.obs.assay, field="name")
tissues = lb.Tissue.from_values(conde22.obs.tissue, field="name")

In [ ]:
file.add_labels(cell_types, "cell_type")
file.add_labels(efs, "assay")
file.add_labels(tissues, "tissue")

In [ ]:
donors = ln.Label.from_values(conde22.obs["donor"])
file.add_labels(donors, feature="donor")

The file is now queryable by everything we linked:

In [ ]:
file.describe()

## A less well curated dataset

### Transform ![](https://img.shields.io/badge/Transform-10b981)

Let's now consider a dataset with less-well curated features:

In [ ]:
pbcm68k = ln.dev.datasets.anndata_pbmc68k_reduced()

We see that this dataset is indexed by gene symbols: 

In [ ]:
pbcm68k.var.index

### Validate ![](https://img.shields.io/badge/Validate-10b981) 

In [ ]:
validated = lb.Gene.validate(pbcm68k.var.index, lb.Gene.symbol)

In this case, we only want to register data with validated genes:

In [ ]:
pbcm68k_validated = pbcm68k[:, validated].copy()

Validate cell types:

In [ ]:
# inspect shows none of the terms are mappable
lb.CellType.inspect(pbcm68k_validated.obs["cell_type"])

# here we search the cell type names from the public ontology and grab the top match
# then add the cell type names from the pbcm68k as synonyms
celltype_bt = lb.CellType.bionty()
ontology_ids = []
mapper = {}
for ct in pbcm68k_validated.obs["cell_type"].unique():
    ontology_id = celltype_bt.search(ct).iloc[0].ontology_id
    record = lb.CellType.from_bionty(ontology_id=ontology_id)
    mapper[ct] = record.name
    record.save()
    record.add_synonym(ct)

pbcm68k_validated.obs["cell_type"] = pbcm68k_validated.obs["cell_type"].map(mapper)

Now, all cell types are validated:

In [ ]:
lb.CellType.validate(pbcm68k_validated.obs["cell_type"]);

### Register ![](https://img.shields.io/badge/Register-10b981) 

In [ ]:
file = ln.File.from_anndata(
    pbcm68k_validated, description="10x reference pbmc68k", var_ref=lb.Gene.symbol
)

In [ ]:
file.save()

In [ ]:
cell_types = lb.CellType.from_values(pbcm68k_validated.obs["cell_type"], "name")
file.add_labels(cell_types, "cell_type")

In [ ]:
file.add_labels(lb.settings.species, feature="species")
file.add_labels(lb.ExperimentalFactor.filter(name=scrna_name).one(), feature="assay")

In [ ]:
var_feature_set = file.features.get_feature_set("var")
var_feature_set.modality = modality
var_feature_set.save()

In [ ]:
file.features

In [ ]:
file.describe()

In [ ]:
file.view_lineage()

🎉 Now let's continue with data integration: {doc}`./scrna2`